# PART 2

In [29]:
import os
import io
import random
from google.cloud import bigquery
from google.oauth2 import service_account
import sqlite3

In [30]:
service_path = "/Users/Marcus/Wedge_Final/"
service_file = 'umt-msba-290c4d48a8fb.json'
gbq_proj_id = 'umt-msba'
gbq_dataset_id = 'wedge_transactions'

private_key =service_path + service_file

In [31]:
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [32]:
query_card_nos = """SELECT distinct card_no
    from `umt-msba.wedge_transactions.transArchive*`
    where card_no !=3"""
query_get_card_nos = client.query(query_card_nos,location ='US',)

In [5]:
owners = []

for idx, row in enumerate(query_get_card_nos) :
    card_no = row[0]
    
    owners.append(card_no)

print(len(owners))

27207


In [18]:
random.seed(294)
samp_size = 270
samp_owners = random.choices(owners, k=samp_size)
print(len(samp_owners))

270


In [19]:
owners_join = ",".join([str(owner) for owner in samp_owners])
query = """Select * From `umt-msba.wedge_transactions.transArchive*` Where card_no in (""" + owners_join + ")"
query_own = client.query(query,location ='US',)

In [20]:
headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"]
with open ("sample_of_owners.txt", "w") as outfile :
    outfile.write ("\t".join(headers) + "\n")
    for row in query_own:
        outfile.write("\t".join([str(item) for item in row]) + "\n")

# Part 3

## Sales by date by hour

In [33]:
#By calendar date (YYYY-MM-DD) and hour of the day, 
#determine the total spend in the store, 
#the number of transactions, 
#and a count of the number of items . 

query=('''SELECT (EXTRACT(date FROM datetime)) AS Date, 
(EXTRACT(hour FROM datetime)) AS Hour,
SUM(total) AS Sales, 
COUNT(DISTINCT(Date(datetime) || trans_no || emp_no || register_no)) AS Transactions,
SUM(CASE WHEN(trans_status = 'R' OR trans_status = 'V') THEN -1 ELSE 1 END) as Items
FROM `umt-msba.wedge_transactions.transArchive*`
WHERE card_no != 3
AND department != 0
AND department != 15
AND trans_status != 'M'
AND trans_status != 'C'
AND trans_status != 'J'
AND (trans_status = '' 
OR trans_status = ' ' 
OR trans_status =  'V' 
OR trans_status = 'R') 
GROUP BY Date, Hour
ORDER BY Date, Hour''')

query_sales_date_hour = client.query(query,location ='US',)

In [34]:
headers = ["date", "hours", "sales", "transactions", "items"]
with open ("sales_date_hour.txt", "w") as outfile :
    outfile.write ("\t".join(headers) + "\n")
    for row in query_sales_date_hour:
        newlist = [str(item) for item in row]
        outfile.write("\t".join(newlist) + "\n")

In [9]:
import sqlite3
db = sqlite3.connect("Welnel_Wedge.db")
cur = db.cursor()

In [23]:
#create table in database

input_txt = "sales_date_hour.txt"
cur.execute('''DROP TABLE IF EXISTS sales_date_hour''')
cur.execute('''CREATE TABLE sales_date_hour (
    date TIMESTAMP, 
    hour INTEGER, 
    sales INTEGER,
    transactions INTEGER,
    items INTEGER)''')

with open(input_txt,'r', encoding="Latin-1") as inputfile :
    next(inputfile)
    for idx, line in enumerate(inputfile.readlines()) :
        line = line.strip().split("\t")
        cur.execute('''
            INSERT INTO sales_date_hour (date,hour,sales,transactions,items)
            VALUES (?,?,?,?,?)''', line)

db.commit()

## Sales by owner by year by month

In [12]:
# A file that has the following columns: card_no, year, month, sales, transactions, and items.

query = ('''SELECT card_no As Owner,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || trans_no || emp_no || register_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'R' OR trans_status = 'V') THEN -1 ELSE 1 END) as Items
    FROM `umt-msba.wedge_transactions.transArchive*`
    WHERE card_no != 3
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY Owner, Year, Month
    ORDER BY Owner, Year, Month DESC''')


query_owner_year_month = client.query(query,location ='US',)

In [14]:
headers = ["owner","year","month","sales","transactions","items"]
with open ("owner_year_month.txt", "w") as outfile :
    outfile.write ("\t".join(headers) + "\n")
    for row in query_owner_year_month:
        newlist = [str(item) for item in row]
        outfile.write("\t".join(newlist) + "\n")

In [15]:
#create table in database

input_txt = "sales_by_owner_by_year_by_month.txt"
cur.execute('''DROP TABLE IF EXISTS owner_year_month''')
cur.execute('''CREATE TABLE owner_year_month (
    owner INTEGER, year INTEGER, month INTEGER, sales INTEGER, transactions INTEGER, items INTEGER)''')

with open(input_txt,'r', encoding="Latin-1") as inputfile :
    next(inputfile)
    for idx, line in enumerate(inputfile.readlines()) :
        line = line.strip().split("\t")
        cur.execute('''
            INSERT INTO owner_year_month (owner,year,month,sales,transactions,items)
            VALUES (?,?,?,?,?,?)''', line)
db.commit()

## Sales by product description by year by month

In [17]:
# A file that has the following columns: 
# upc, description, department number, department name, year, month, sales, transactions, and items.

query = ('''SELECT deplook.dept_name, trans.department, description,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    upc,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || trans_no || emp_no || register_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'R' OR trans_status = 'V') THEN -1 ELSE 1 END) as Items,
    trans.department AS dept_no
    FROM `umt-msba.wedge_transactions.transArchive*` AS trans
    LEFT OUTER JOIN `umt-msba.wedge_transactions.department_lookup` AS deplook ON trans.department = deplook.department
    WHERE card_no != 3
    AND trans.department != 0 
    AND trans.department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY Year, Month, upc, description, dept_no, deplook.dept_name
    ORDER BY description, Year, Month DESC''')

query_sales_prod_year_month = client.query(query,location ='US',)

In [19]:
headers = ["department","dept_name","description", "year","month", "upc", "sales", "transactions", "items","dept_no"]
with open ("sales_prod_year_month.txt", "w") as outfile :
    outfile.write ("\t".join(headers) + "\n")
    for row in query_sales_prod_year_month:
        newlist = [str(item) for item in row]
        outfile.write("\t".join(newlist) + "\n")

In [20]:
#create table in database

input_txt = "sales_prod_year_month.txt"
cur.execute('''DROP TABLE IF EXISTS sales_prod_year_month''')
cur.execute('''CREATE TABLE sales_prod_year_month (
    Department INTEGER,
    Dept_name INTERGER,
    Description INTERGER,
    Year INTEGER,
    Month INTEGER,
    upc INTEGER,
    Sales INTEGER,
    Transactions INTEGER,
    Items INTEGER,
    dept_no INTEGER)''')

with open(input_txt,'r', encoding="Latin-1") as inputfile :
    next(inputfile)
    for idx, line in enumerate(inputfile.readlines()) :
        line = line.strip().split("\t")
        cur.execute('''
            INSERT INTO sales_prod_year_month (Department,Dept_name,Description,Year,Month,upc,Sales,Transactions,Items,dept_no)
            VALUES (?,?,?,?,?,?,?,?,?,?)''', line)

db.commit()